# pix2pix

In [ ]:
import tensorflow as tf
import numpy as np
import glob

BATCH_SIZE = 1

# TODO: Disect
def conv(batch_input, out_channels, stride):
    in_channels = batch_input.get_shape()[3]
    filter = tf.get_variable("filter", [4, 4, in_channels, out_channels], dtype=tf.float32, initializer=tf.random_normal_initializer(0, 0.02))
    # [batch, in_height, in_width, in_channels], [filter_width, filter_height, in_channels, out_channels]
    #     => [batch, out_height, out_width, out_channels]
    padded_input = tf.pad(batch_input, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
    conv = tf.nn.conv2d(padded_input, filter, [1, stride, stride, 1], padding="VALID")
    return conv

## Load data ##
input_paths = glob.globa('data/pix2pix/facades/train/*.jpg')
decode = tf.image_decode_jpeg
path_queue = tf.train.string_input_producer(input_paths, shuffle=True)
reader = tf.WholeFileReader()
paths, contents = read.read(path_queue)
rawInput = decode(contents)
rawInput = tf.image.convert_image_dtype(rawInput, dtype=tf.float32)

# [height, width, channel]
raw_input.set_shape([None, None, 3])
width = tf.shape(rawInput)[1]

preprocess = lambda x: x * 2 - 1 # [0, 1] => [-1, 1]
a_images = preprocess(rawInput[:,:width//2,:])
b_images = preprocess(rawInput[:,width//2:,:])

# Can be switched for other dir
inputs, targets = [a_images, b_images]

paths, inputs, targets = tf.train.batch([paths, inputs, targets], batch_size=BATCH_SIZE)
steps_per_epoch = int(math.ceil(len(input_paths) / a.batch_size))

out_channels = int(targets.get_shape()[-1])

## Create Generator ##
output = conv(generator_inputs, 64, stide=2)
layers.append(output)